In [151]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
df = pd.read_csv('data/Index_of_Place_Names_in_Great_Britain_(July_2016).csv')

In [21]:
df.columns

Index(['X', 'Y', 'objectid_1', 'objectid', 'placeid', 'place15cd', 'placesort',
       'place15nm', 'splitind', 'popcnt', 'descnm', 'ctyhistnm', 'ctyltnm',
       'ctry15nm', 'cty15cd', 'cty15nm', 'lad15cd', 'lad15nm', 'laddescnm',
       'wd15cd', 'par15cd', 'hlth12cd', 'hlth12nm', 'regd15cd', 'regd15nm',
       'rgn15cd', 'rgn15nm', 'npark15cd', 'npark15nm', 'bua11cd', 'pcon15cd',
       'pcon15nm', 'eer15cd', 'eer15nm', 'pfa15cd', 'pfa15nm', 'gridgb1m',
       'gridgb1e', 'gridgb1n', 'grid1km', 'lat', 'long_'],
      dtype='object')

In [4]:
df.head(10)

,X,Y,objectid_1,objectid,placeid,place15cd,placesort,place15nm,splitind,popcnt,...,eer15cd,eer15nm,pfa15cd,pfa15nm,gridgb1m,gridgb1e,gridgb1n,grid1km,lat,long_
0,127213,805291,1,1,382,IPN0000419,achill,A' Chill,0,,...,S15000001,Scotland,S23000009,Scotland,1272130805291,127213,805291,NG2705,57.057645,-6.500824
1,238625,725335,2,2,17379,IPN0018256,crianlarich,A' Chrìon Làraich,0,,...,S15000001,Scotland,S23000009,Scotland,2386250725335,238625,725335,NN3825,56.392640,-4.615908
2,118024,850126,3,3,26626,IPN0027987,glasphein,A' Glas Pheighinn,0,,...,S15000001,Scotland,S23000009,Scotland,1180240850126,118024,850126,NG1850,57.453522,-6.702790
3,167998,768309,4,4,46448,IPN0048777,moss,A' Mhointeach,0,,...,S15000001,Scotland,S23000009,Scotland,1679980768309,167998,768309,NM6768,56.748497,-5.796153
4,164719,823108,5,5,6291,IPN0006615,blackpark,A' Phairce Dhubh,0,,...,S15000001,Scotland,S23000009,Scotland,1647190823108,164719,823108,NG6423,57.238000,-5.900764
5,209521,776848,6,6,16465,IPN0017294,corpach,A'Chorpaich,0,,...,S15000001,Scotland,S23000009,Scotland,2095210776848,209521,776848,NN0976,56.843908,-5.124655
6,495783,143522,7,7,1,IPN0000001,aaronshill,Aaron's Hill,0,,...,E15000008,South East,E23000031,Surrey,4957830143522,495783,143522,SU9543,51.182914,-0.630976
7,473638,323718,8,8,325,IPN0000356,abkettleby,Ab Kettleby,0,529,...,E15000004,East Midlands,E23000021,Leicestershire,4736380323718,473638,323718,SK7323,52.805997,-0.909079
8,401400,251700,9,9,326,IPN0000357,ablench,Ab Lench,0,,...,E15000005,West Midlands,E23000016,West Mercia,4014000251700,401400,251700,SP0151,52.163569,-1.980949
9,306727,270918,10,10,39,IPN0000039,abbeycwmhir,Abaty Cwm-hir,0,235,...,W08000001,Wales,W15000004,Dyfed-Powys,3067270270918,306727,270918,SO0670,52.328398,-3.370117


In [257]:
trim_df = df[['place15nm', 'lat', 'long_']]

In [258]:
trim_df.columns = ['place_names', 'lat', 'long']

In [236]:
def processer(origin, places):
    
    def term_matching(term, row, string_match):
        places_matching_term = places[places.place15nm_sp.str.contains(string_match)].place15nm
        for place in places_matching_term:
            check = True
            for previous_term in place_tag_dict[place][0]:    #if 'by' is matched, won't match 'y'
                if term in previous_term:
                    check = False
            if check:
                place_tag_dict[place][0].append(term)
                for tag in row[2].split(','):
                    if tag != '':
                        place_tag_dict[place][1].append(tag)
    
    place_tag_dict = {key:([],[]) for key in places.place15nm}
    
    places['place15nm_sp'] = places.place15nm.map(lambda x: ' ' + x.strip() + ' ')
    
    for row in tqdm(origin.itertuples()):
        for term in sorted(row[1].split(','), key=len, reverse=True):
            if type(row[3]) == float:
                term_matching(term, row, term)
                continue
            fix_dict = {'suffix': r'\w+'+ term + r'\W', 
                        'prefix': r'\W' + term + r'\w+', 
                        'interfix': r'[-\s]{}[-\s]'.format(term)}
            for fix in fix_dict.keys():
                if fix in row[3]:
                    term_matching(term, row, fix_dict[fix])
                    break
    place_tag_dict.keys()
    return place_tag_dict



In [7]:
def strip(entry):
    try:
        new_entry = []
        for char in entry:
            if char == ',':
                new_entry.append(char)
            elif char.isalpha():
                new_entry.append(char)
        new_entry = ''.join(new_entry)
        return new_entry
    except:
        return entry

def striplower(entry):
    try:
        new_entry = []
        for char in entry:
            if char == ',':
                new_entry.append(char)
            elif char.isupper():
                new_entry.append(char)
        new_entry = ''.join(new_entry)
        return new_entry
    except:
        return entry

In [244]:
csv = pd.read_clipboard(sep=',')
csv.to_csv('data/table-1.csv')

In [245]:
origins = pd.read_csv('data/table-1.csv')

origins = origins.applymap(lambda x: strip(x))

origins = origins[['Term', 'Origin', 'Position']]

origins.Origin = origins.Origin.map(lambda x: striplower(x))

origins['Origin'] = origins.Origin.astype('string')

In [246]:
place_tag_dict = processer(origins, df)

139it [00:12, 11.06it/s]


In [247]:
place_tag_df = pd.DataFrame().from_dict(place_tag_dict, orient='index', columns=['terms', 'tags'])

In [248]:
place_tag_df.reset_index(inplace=True)

In [249]:
mlb = MultiLabelBinarizer()
dummy_df = pd.DataFrame(mlb.fit_transform(place_tag_df.tags), columns=mlb.classes_)

In [251]:
place_dummies_df = pd.merge(place_tag_df, dummy_df, left_index=True, right_index=True)

In [255]:
place_dummies_df.columns = ['place_names', 'terms', 'tags', 'B', 'C', 'I', 'K', 'KL', 'L', 'ME', 'NF','OE', 'ON', 'P', 'S', 'SG', 'W']

In [261]:
final_df = pd.merge(place_dummies_df, trim_df, on='place_names')

In [264]:
final_df['tags'] = final_df.tags.map(lambda x: np.nan if len(x) == 0 else x)

In [267]:
final_df = final_df.dropna().copy()

In [268]:
final_df.to_csv('final_df.csv')